### Importing the packages that we need to conduct necesary cleaning and streamlit.

In [1]:
# Import necessary packages
import os
import pandas as pd
import re
import pdfplumber
import streamlit as st
import matplotlib.pyplot as plt

### Importing all AMEX Statements and Cleaning Data to Desired Table Format 
#### Description, Amount, Category, Month, Day, Action

Actions are powered by a categorization mapping .CSV file.

In [2]:
# American Express Bank
folder_path = "/Users/addakinthomas/Desktop/Finances/Addakin/amex/"
dfs = []

for file in os.listdir(folder_path):
    if file.endswith(".csv"):
        file_path = os.path.join(folder_path, file)

        month_name = file.split("_")[0]

        df = pd.read_csv(file_path)

        dfs.append(df)

df= pd.concat(dfs,ignore_index=True)
df['Date'] = pd.to_datetime(df['Date'], errors='coerce', format='%m/%d/%Y')

# Extract Month Name and Day
df['Month'] = df['Date'].dt.strftime('%B')  # "January"
df['Day'] = df['Date'].dt.day  # Extracts day as an integer (1-31)
df['Action'] = df['Amount'].apply(lambda x: 'Spend' if x > 0 else 'Income')

# Read the spending categories file
spending_categories = pd.read_csv("/Users/addakinthomas/Desktop/Finances/Addakin/spending_categories.csv")

# Create a dictionary for mapping
category_mapping = dict(zip(spending_categories['name'], spending_categories['category']))

# Map the 'Category' column in df to the new categories
df['Category'] = df['Category'].map(category_mapping)
df['Statement'] = 'AMEX'

# Drop not needed column
df1 = df.drop(columns=['Date', 'Account #', 'Card Member', 'Extended Details', 'Reference', 'Appears On Your Statement As', 'Address', 'City/State', 'Zip Code', 'Country'])

# Display final DataFrame
print(df1)


                                    Description  Amount    Category     Month  \
0                                          UBER   16.39        Uber  February   
1                                          UBER   20.93        Uber  February   
2    Interest Charge on Pay Over Time Purchases   43.88        Fees  February   
3    TST* MIKE'S BEER BARPITTSBURGH          PA    3.67         Bar  February   
4                                          UBER   14.96        Uber  February   
..                                          ...     ...         ...       ...   
165  AplPay STARBUCKS    800-782-7282        WA   10.00         Bar  December   
166  THE MERCHANT KITCHENBOSTON              MA   53.50  Restaurant  December   
167  TST* CRAFT FOOD HALLBOSTON              MA   19.14  Restaurant  December   
168  TST* TATTE BAKERY ONBOSTON              MA   14.04  Restaurant  December   
169  MA MAISON 6500000034BOSTON              MA   85.00  Restaurant  December   

     Day Action Statement  

### df1 outputted, cleaned AMEX table
### next step is to do the same to Chase bank statements

In [3]:
import pdfplumber
import re
import pandas as pd
import os

# Define the folder containing the PDF files
folder_path = "/Users/addakinthomas/Desktop/Finances/Addakin/chase/"

# Initialize an empty list to store all transactions across multiple PDFs
all_transactions = []

# Loop through all PDF files in the folder
for filename in os.listdir(folder_path):
    if filename.endswith(".pdf"):  # Ensure we only process PDF files
        pdf_path = os.path.join(folder_path, filename)
        print(f"Processing: {filename}")

        # Open the PDF
        with pdfplumber.open(pdf_path) as pdf:
            num_pages = len(pdf.pages)  # Get total number of pages
            start_page = 1  # Page index for second page (0-based index)
            end_page = num_pages - 1  # 2nd to last page

            for page_num in range(start_page, end_page):  # Loop through selected pages
                page = pdf.pages[page_num]
                text = page.extract_text()

                if text:
                    # **Remove extra text before and after the transaction list**
                    text = re.sub(r".*?\*start\*transaction detail", "", text, flags=re.DOTALL)  # Remove everything before "*start*transaction detail"
                    text = re.sub(r"\*end\*transac.*", "", text, flags=re.DOTALL)  # Remove everything after "*end*transac"
                    text = text.strip()  # Clean up leading/trailing whitespace

                    # Regular expression to match transaction entries
                    pattern = re.compile(
                        r"(\d{2}/\d{2})\s+(.+?)\s+([-]?\d{1,3}(?:,\d{3})*(?:\.\d{2})?)\s+(\d{1,3}(?:,\d{3})*(?:\.\d{2})?)"
                    )

                    # Extract transactions using regex
                    transactions = pattern.findall(text)

                    # Append transactions to the list, adding the file name for reference
                    for transaction in transactions:
                        all_transactions.append((filename,) + transaction)

# Convert extracted data into a DataFrame
df = pd.DataFrame(all_transactions, columns=["File", "Date", "Description", "Amount", "Balance"])

# Convert Amount and Balance to numeric values
df["Amount"] = df["Amount"].str.replace(",", "").astype(float)
df["Balance"] = df["Balance"].str.replace(",", "").astype(float)

# Display the DataFrame

df["Description"] = df["Description"]
df["Amount"] = df["Amount"]
df['Category'] = df['Description'].map(category_mapping).fillna('Other')
df['Date'] = pd.to_datetime(df['Date'], format='%m/%d')
df['Month'] = df['Date'].dt.strftime('%B')  # "January"
df['Day'] = df['Date'].dt.day  # Extracts day as an integer (1-31)
df['Action'] = df['Amount'].apply(lambda x: 'Spend' if x < 0 else 'Income')
df['Amount'] = df['Amount'].abs()
df['Statement'] = 'CHASE'
df = df.drop(columns=['File', 'Date', 'Balance'])

df2 = df
print(df)

Processing: january_2025.pdf
Processing: february_2025.pdf
                                          Description   Amount Category  \
0   American Express ACH Pmt M8014 Web ID: 2005032111    23.20    Other   
1                               Remote Online Deposit     1.00    Other   
2      Venmo Payment 1039105423155 Web ID: 3264681992    40.00    Other   
3      Venmo Payment 1039105425873 Web ID: 3264681992    20.00    Other   
4   Card Purchase 12/21 Draftkings 6179866744 MA C...    10.00    Other   
..                                                ...      ...      ...   
90  Card Purchase 02/16 Betrivers PA 8555851401 IL...    25.00    Other   
91  Card Purchase 02/16 Betrivers PA 8555851401 IL...   100.00    Other   
92  Payment Sent 02/18 Venmo *Will Dimond Visa Dir...  1682.00    Other   
93          Tpg Products Sbtpg LLC PPD ID: 3722260102    84.98    Other   
94         Betrivers PA Viatrustly Web ID: 1218965010   125.00    Other   

       Month  Day  Action Statement  
0 

/var/folders/dq/yjshrdhd0l1_csn5j_xbm77h0000gn/T/ipykernel_14018/39363057.py:58: DeprecationWarning: Parsing dates involving a day of month without a year specified is ambiguious
and fails to parse leap day. The default behavior will change in Python 3.15
to either always raise an exception or to use a different default year (TBD).
To avoid trouble, add a specific year to the input & format.
See https://github.com/python/cpython/issues/70647.
  df['Date'] = pd.to_datetime(df['Date'], format='%m/%d')


### outputted df2 from chase bank, now union the tables together

In [4]:
finances = pd.concat([df1, df2], ignore_index=False)
print(finances)

folder_path = "/Users/addakinthomas/Desktop/Finances/Addakin/streamlit/"
df.to_csv(os.path.join(folder_path, "finances.csv"), index=False) 

                                          Description   Amount Category  \
0                                                UBER    16.39     Uber   
1                                                UBER    20.93     Uber   
2          Interest Charge on Pay Over Time Purchases    43.88     Fees   
3          TST* MIKE'S BEER BARPITTSBURGH          PA     3.67      Bar   
4                                                UBER    14.96     Uber   
..                                                ...      ...      ...   
90  Card Purchase 02/16 Betrivers PA 8555851401 IL...    25.00    Other   
91  Card Purchase 02/16 Betrivers PA 8555851401 IL...   100.00    Other   
92  Payment Sent 02/18 Venmo *Will Dimond Visa Dir...  1682.00    Other   
93          Tpg Products Sbtpg LLC PPD ID: 3722260102    84.98    Other   
94         Betrivers PA Viatrustly Web ID: 1218965010   125.00    Other   

       Month  Day  Action Statement  
0   February    9   Spend      AMEX  
1   February    9   Spe

### outputted df finances to power streamlit application

use %%writefile app.py

In [ ]:
%%writefile app.py
import streamlit as st
import pandas as pd
import os
import matplotlib.pyplot as plt

# Define file path dynamically
csv_file = ""
feedback_folder = "feedback"
feedback_file = os.path.join(feedback_folder, "feedback.txt")

# Load CSV data safely
def load_data():
    try:
        df = pd.read_csv(csv_file)
        df.columns = df.columns.str.lower().str.strip()  # Normalize column names to lowercase and remove spaces
        return df
    except FileNotFoundError:
        return None

# Create feedback directory if not exists
if not os.path.exists(feedback_folder):
    os.makedirs(feedback_folder)

# Streamlit UI
st.set_page_config(layout="wide")
st.title("📊 Addakin's Financial Overview")

df = load_data()

if df is None:
    st.error(f"❌ CSV file not found at: `{csv_file}`")
else:
    months_ordered = ["December", "January", "February"]
    available_months = [month for month in months_ordered if month.lower() in df['month'].dropna().str.lower().unique()]
    
    # Sidebar Filters
    st.sidebar.header("📅 Filter by Month")
    selected_month = st.sidebar.radio("Select a month", available_months)
    if selected_month:
        df = df[df['month'].str.lower() == selected_month.lower()]
    
    # Categorize "Other"
    df.loc[df['category'].str.lower() == 'other', 'category'] = 'Other'
    uncategorized_df = df[df['category'] == 'Other']
    df = df[df['category'] != 'Other']
    
    # Tabs
    tab1, tab2, tab3, tab4 = st.tabs(["💸 Spending", "💰 Saving", "📈 Income", "❓ Uncategorized Transactions"])
    
    with tab1:
        st.subheader("📊 Spending Transactions")
        spending_df = df[df['action'].str.lower() == 'spend']
        if not spending_df.empty:
            st.dataframe(spending_df)
        else:
            st.warning("⚠️ No spending transactions found.")
    
    with tab2:
        st.subheader("💰 Saving Transactions")
        saving_df = df[df['action'].str.lower() == 'save']
        if not saving_df.empty:
            st.dataframe(saving_df)
        else:
            st.warning("⚠️ No saving transactions found.")
    
    with tab3:
        st.subheader("📈 Income Transactions")
        income_df = df[df['action'].str.lower() == 'income']
        if not income_df.empty:
            st.dataframe(income_df)
        else:
            st.warning("⚠️ No income transactions found.")
    
    with tab4:
        st.subheader("❓ Uncategorized Transactions")
        if not uncategorized_df.empty():
            st.dataframe(uncategorized_df)
        else:
            st.warning("⚠️ No uncategorized transactions found.")
    
    # Feedback Section
    st.sidebar.subheader("💡 Feedback & Suggestions")
    feedback_text = st.sidebar.text_area("How can we improve this dashboard?")
    if st.sidebar.button("Submit Feedback"):
        with open(feedback_file, "a") as f:
            f.write(feedback_text + "\n")
        st.sidebar.success("✅ Thank you for your feedback!")


Overwriting app.py
